In [1]:
import os
import sys

import plotly.graph_objects as go

from ray import tune
from ray.tune.schedulers import ASHAScheduler
from torch.optim.lr_scheduler import ReduceLROnPlateau

#TODO: Why is this needed? os.environ['PYTHONPATH'] = module_path
module_path = os.path.abspath(os.path.join(os.getcwd(), ".."))
if module_path not in sys.path:
    os.environ['PYTHONPATH'] = module_path
    sys.path.append(module_path)

from src.train import train_model

In [2]:
target_variable = 'Flow_Kalltveit'
file_name = "cleaned_data_1.csv"
data_dir = "../best_model"
datetime_variable = "Datetime"
models = [
            "LSTM", "LSTMTemporalAttention", "LSTMSpatialTemporalAttention"
         ] # Can be: "FCN", "FCNTemporalAttention", "LSTMTemporalAttention", "LSTM", "LSTMSpatialAttention", "LSTMSpatialTemporalAttention"
variables = [
            [],
            [
                "Wind_Speed_Nilsebu",
                "Air_Temperature_Nilsebu",
                "Wind_Direction_Nilsebu",
                "Relative_Humidity_Nilsebu",
                "Air_Temperature_Fister",
                "Precipitation_Fister",
                "Flow_Lyngsvatn_Overflow",
                "Flow_Tapping",
                "Water_Level_Kalltveit",
                "Water_Temperature_Kalltveit_Kum",
                "Precipitation_Nilsebu",
                "Flow_HBV",
                "Precipitation_HBV",
                "Temperature_HBV",
                "Flow_Without_Tapping_Kalltveit",
                "Flow_Lyngsaana",
                "Water_Temperature_Lyngsaana"
            ],
        ]

In [3]:
def main(exp_name, n_samples, max_num_epochs, min_num_epochs, local_dir="../ray_results/"):
    config = {
        "data_file": file_name,
        "datetime":  datetime_variable,
        
        "data": {
            "target_variable": target_variable,
            "sequence_length": tune.choice([25]),
            "batch_size": tune.choice([256, 512]),
            "variables": tune.grid_search(variables)
        },

        "model": tune.grid_search(models), 
        "model_arch": {
            "input_size": None,
            "hidden_size": tune.choice([32, 64]),
            'num_layers': tune.choice([2, 3, 4]),
            "output_size": 1
        },

        "training": {
            "learning_rate": tune.loguniform(1e-4, 1e-1),
            "weight_decay": tune.choice([0, 0.001, 0.0001]),
        },

        'num_epochs': tune.choice([200]),
    }

    reporter = tune.JupyterNotebookReporter(
            metric_columns=[
                "train_loss", "val_loss", "test_loss", "training_iteration"
            ])

    scheduler = ASHAScheduler(
        max_t=max_num_epochs,
        grace_period=min_num_epochs,
        reduction_factor=2)

    results = tune.run(
        train_model, # TODO: partial(train_cifar, data_dir=data_dir),
        resources_per_trial={"cpu": 12, "gpu": 1},
        config=config,
        num_samples=n_samples,
        scheduler=scheduler,
        progress_reporter=reporter,
        name=exp_name,
        local_dir=local_dir,
        metric='val_loss',
        mode='min',
    )

    return results

In [4]:
def print_best_model(all_trials):
    # Get all trials

    # Sort trials by 'val_loss' in ascending order
    sorted_trials = sorted(all_trials, key=lambda trial: trial.metric_analysis['val_loss']['min'])
    # Initialize a dictionary to store the best trial for each model type
    best_trials_by_model = {}

    # Find the best trials for each model type
    for trial in sorted_trials:
        model_type = trial.config['model']
        if model_type not in best_trials_by_model:
            best_trials_by_model[model_type] = trial

    # Print the best performance for each model type and their validation losses
    print("Best models and their validation losses:")
    for model_type, trial in best_trials_by_model.items():
        val_loss = trial.metric_analysis['val_loss']['min']
        variables = trial.config['data']['variables']
        print(f"{model_type}: Trial {trial.trial_id} - val_loss: {val_loss:.4f} - variables: {variables}")

In [5]:
def plot_best_mode(df):
    best_model = df
    # Create a line chart
    fig = go.Figure()

    # Add training loss trace
    fig.add_trace(go.Scatter(x=best_model.index, y=best_model['train_loss'],
                        mode='lines',
                        name='Training Loss'))

    # Add validation loss trace
    fig.add_trace(go.Scatter(x=best_model.index, y=best_model['val_loss'],
                        mode='lines',
                        name='Validation Loss'))

    # Customize the layout
    fig.update_layout(title='Training and Validation Loss',
                    xaxis_title='Iteration',
                    yaxis_title='Loss',
                    legend_title='Loss Types')
    print("Plot of the best model:")
    # Show the plot
    fig.show()

In [6]:
exp_name = "inflow_forecasting"

analysis = main(exp_name, n_samples=2, max_num_epochs=200, min_num_epochs=100)
# Models trained and last reported loss
analysis.dataframe()[['train_loss', 'val_loss', 'train_loss', 'config/model', 'time_total_s', 'config/data/variables']]

2023-04-04 16:31:46,920	INFO worker.py:1553 -- Started a local Ray instance.
  0%|          | 0/45 [00:00<?, ?it/s]


Trial name,date,done,episodes_total,experiment_id,hostname,iterations_since_restore,node_ip,pid,should_checkpoint,test_loss,time_since_restore,time_this_iter_s,time_total_s,timestamp,timesteps_since_restore,timesteps_total,train_loss,training_iteration,trial_id,val_loss,warmup_time
train_model_6e993_00000,2023-04-04_16-31-55,False,,1e632e311bf748db9023dd4c3ea4541e,DESKTOP-D4IVECG,1,127.0.0.1,32544,True,19.2573,3.96708,3.96708,3.96708,1680618715,0,,19.2573,1,6e993_00000,3.79979,0.00546598


  0%|          | 0/23 [00:00<?, ?it/s]

  0%|          | 0/45 [00:00<?, ?it/s]

  0%|          | 0/10 [00:00<?, ?it/s]
2023-04-04 16:48:43,051	INFO tune.py:798 -- Total run time: 1014.78 seconds (1014.69 seconds for the tuning loop).


,train_loss,val_loss,train_loss,config/model,time_total_s,config/data/variables
0,6.172026,5.416369,6.172026,LSTM,180.509150,[]
1,29.641283,18.805010,29.641283,LSTM,10.186466,"[Wind_Speed_Nilsebu, Air_Temperature_Nilsebu, ..."
2,2.188978,1.681420,2.188978,LSTMTemporalAttention,232.738536,[]
3,0.241249,0.108950,0.241249,LSTMTemporalAttention,22.241445,"[Wind_Speed_Nilsebu, Air_Temperature_Nilsebu, ..."
4,2.954106,1.481803,2.954106,LSTMSpatialTemporalAttention,89.170599,[]
5,0.567592,0.333122,0.567592,LSTMSpatialTemporalAttention,21.694145,"[Wind_Speed_Nilsebu, Air_Temperature_Nilsebu, ..."
6,6.102352,1.924167,6.102352,LSTM,107.251965,[]
7,29.572056,18.410117,29.572056,LSTM,12.168606,"[Wind_Speed_Nilsebu, Air_Temperature_Nilsebu, ..."
8,4.711717,4.519623,4.711717,LSTMTemporalAttention,122.186541,[]
9,2.297323,0.965194,2.297323,LSTMTemporalAttention,22.834173,"[Wind_Speed_Nilsebu, Air_Temperature_Nilsebu, ..."


In [7]:
print_best_model(analysis.trials)
plot_best_mode(analysis.best_dataframe)

Best models and their validation losses:
LSTMTemporalAttention: Trial 6e993_00003 - val_loss: 0.0713 - variables: ['Wind_Speed_Nilsebu', 'Air_Temperature_Nilsebu', 'Wind_Direction_Nilsebu', 'Relative_Humidity_Nilsebu', 'Air_Temperature_Fister', 'Precipitation_Fister', 'Flow_Lyngsvatn_Overflow', 'Flow_Tapping', 'Water_Level_Kalltveit', 'Water_Temperature_Kalltveit_Kum', 'Precipitation_Nilsebu', 'Flow_HBV', 'Precipitation_HBV', 'Temperature_HBV', 'Flow_Without_Tapping_Kalltveit', 'Flow_Lyngsaana', 'Water_Temperature_Lyngsaana']
LSTMSpatialTemporalAttention: Trial 6e993_00011 - val_loss: 0.1129 - variables: ['Wind_Speed_Nilsebu', 'Air_Temperature_Nilsebu', 'Wind_Direction_Nilsebu', 'Relative_Humidity_Nilsebu', 'Air_Temperature_Fister', 'Precipitation_Fister', 'Flow_Lyngsvatn_Overflow', 'Flow_Tapping', 'Water_Level_Kalltveit', 'Water_Temperature_Kalltveit_Kum', 'Precipitation_Nilsebu', 'Flow_HBV', 'Precipitation_HBV', 'Temperature_HBV', 'Flow_Without_Tapping_Kalltveit', 'Flow_Lyngsaana', '